In [2]:
import os
import pathlib
import torch
import spacy
from OIE.datasets.validated_splits.contractions import transform_portuguese_contractions, clean_extraction
from OIE.final.matcher import OIE_Match
import pandas as pd
import numpy as np

In [2]:
torch.cuda.is_available()

True

In [3]:
class ArgsRel:
    def __init__(self):
        self.current_root_sint = None
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")

    def root_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "VERB" and dep == "ROOT") and (idx != 0 and idx != len(doc_dict) - 1):
                root_idx = (idx, idx)
                self.current_root_sint = "VERB-ROOT"
                break
        if root_idx == (None, None):
            root_idx = self.aux_parse(doc_dict, root_idx)
        return root_idx

    def aux_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "AUX" and dep == "cop") and (idx != 0 and idx != len(doc_dict) - 1):
                root_idx = (idx, idx)
                self.current_root_sint = "AUX-cop"
                break
            elif (pos == "AUX" and dep == "ROOT") and (idx != 0 and idx != len(doc_dict) - 1):
                root_idx = (idx, idx)
                self.current_root_sint = "AUX-ROOT"
                break
        if root_idx == (None, None):
            root_idx = self.x_comp_parse(doc_dict, root_idx)
        return root_idx
    def x_comp_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "VERB" and dep == "xcomp" and (idx != 0 and idx != len(doc_dict) - 1)):
                root_idx = (idx, idx)
                self.current_root_sint = "VERB-xcomp"
                break
            elif (pos == "VERB" and dep == "acl" and (idx != 0 and idx != len(doc_dict) - 1)):
                root_idx = (idx, idx)
                self.current_root_sint = "VERB-acl"
                break
            elif (pos == "VERB" and dep == "acl:relcl" and (idx != 0 and idx != len(doc_dict) - 1)):
                root_idx = (idx, idx)
                self.current_root_sint = "VERB-acl:relacl"
                break
        if root_idx == (None, None):
            root_idx = self.noun_root_parse(doc_dict, root_idx)
        return root_idx

    def noun_root_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "NOUN" and dep == "ROOT" and (idx != 0 and idx != len(doc_dict) - 1)):
                root_idx = (idx, idx)
                self.current_root_sint = "NOUN-ROOT"
                break
        return root_idx

    def get_args_rel(self, ext):
        doc = self.nlp(ext)
        doc_dict = {}
        i = 0
        for token in doc:
            doc_dict[i] = {"text": token.text, "pos": token.pos_, "dep": token.dep_}
            i += 1
        arg1 = ""
        rel = ""
        arg2 = ""
        root_idx = (None, None)
        self.current_root_sint = None
        root_idx = self.root_parse(doc_dict, root_idx)
        print(self.current_root_sint)


        #verificando elementos que compoem a rel antes do centro
        if root_idx != (None, None):
            before_root_pos_dep = ""
            print(root_idx[0])
            for i in range(0, root_idx[0]):
                pos = doc_dict[i]["pos"]
                dep = doc_dict[i]["dep"]
                before_root_pos_dep += pos + "-" + dep + ", "
            before_root_pos_dep = before_root_pos_dep[:-2]
            splited = before_root_pos_dep.split(", ")

            if self.current_root_sint == "NOUN-ROOT":
                if "PRON-expl" in before_root_pos_dep and splited[-1] == "PRON-expl":
                    if root_idx[0]-1 > 0:
                        root_idx = (root_idx[0]-1, root_idx[1])
                    else:
                        root_idx = (root_idx[0], root_idx[1])

            if "AUX-cop" in before_root_pos_dep and splited[-1] == "AUX-cop":
                if root_idx[0]-1 > 0:
                    root_idx = (root_idx[0]-1, root_idx[1])
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "AUX-cop, ADV-advmod" in before_root_pos_dep and splited[-1] == "ADV-advmod":
                if root_idx[0]-2 > 0:
                    root_idx = (root_idx[0]-2, root_idx[1])
                else:
                    root_idx = (root_idx[0]-1, root_idx[1])
            elif "ADV-advmod" in before_root_pos_dep and splited[-1] == "ADV-advmod":
                if root_idx[0]-1 > 0:
                    root_idx = (root_idx[0]-1, root_idx[1])
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "AUX-aux" in before_root_pos_dep and splited[-1] == "AUX-aux":
                if root_idx[0]-1 > 0:
                    root_idx = (root_idx[0]-1, root_idx[1])
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "AUX-aux:pass" in before_root_pos_dep and splited[-1] == "AUX-aux:pass":
                if root_idx[0]-1 > 0:
                    root_idx = (root_idx[0]-1, root_idx[1])
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "AUX-aux:pass" in before_root_pos_dep and splited[-1] == "AUX-aux:pass":
                if root_idx[0]-1 > 0:
                    root_idx = (root_idx[0]-1, root_idx[1])
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "ADV-advmod, PRON-obj" in before_root_pos_dep and splited[-1] == "PRON-obj":
                if root_idx[0]-2 > 0:
                    root_idx = (root_idx[0]-2, root_idx[1])
                else:
                    root_idx = (root_idx[0]-1, root_idx[1])
            elif "AUX-cop, ADP-case" in before_root_pos_dep and splited[-1] == "ADP-case":
                if root_idx[0]-2 > 0:
                    root_idx = (root_idx[0]-2, root_idx[1])
                else:
                    root_idx = (root_idx[0]-1, root_idx[1])
            elif "AUX-cop, DET-det" in before_root_pos_dep and splited[-1] == "DET-det":
                if root_idx[0]-2 > 0:
                    root_idx = (root_idx[0]-2, root_idx[1])
                else:
                    root_idx = (root_idx[0]-1, root_idx[1])

        #verificando elementos que compoem a rel depois do centro
        if root_idx != (None, None):
            after_root_pos_dep = ""
            for i in range(root_idx[1]+1, len(doc_dict)):
                pos = doc_dict[i]["pos"]
                dep = doc_dict[i]["dep"]
                after_root_pos_dep += pos + "-" + dep + ", "
            after_root_pos_dep = after_root_pos_dep[:-2]
            splited = after_root_pos_dep.split(", ")

            if self.current_root_sint == "AUX-cop":
                if "DET-det, NOUN-ROOT, ADJ-amod, ADP-case" in after_root_pos_dep and splited[0] == "DET-det":
                    if root_idx[1]+4 < len(doc_dict) - 1:
                        root_idx = (root_idx[0], root_idx[1]+4)
                    else:
                        root_idx = (root_idx[0], root_idx[1])
            if "ADP-case, DET-det, ADV-obl, VERB-xcomp" in after_root_pos_dep and splited[0] == "ADP-case":
                if root_idx[1]+4 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+4)
                else:
                    root_idx = (root_idx[0], root_idx[1]+3)
            elif "ADJ-amod, ADP-case" in after_root_pos_dep and splited[0] == "ADJ-amod":
                if root_idx[1]+2 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+2)
                else:
                    root_idx = (root_idx[0], root_idx[1]+1)
            elif "VERB-xcomp, DET-det, NOUN-obj, ADP-case" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+4 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+4)
                else:
                    root_idx = (root_idx[0], root_idx[1]+3)
            elif "VERB-xcomp, SCONJ-mark, VERB-xcomp, ADP-case" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+4 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+4)
                else:
                    root_idx = (root_idx[0], root_idx[1]+3)
            elif "VERB-xcomp, ADP-case" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+2 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+2)
                else:
                    root_idx = (root_idx[0], root_idx[1]+1)
            elif "VERB-xcomp, VERB-xcomp" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+2 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+2)
                else:
                    root_idx = (root_idx[0], root_idx[1]+1)
            elif "VERB-xcomp, SCONJ-mark, VERB-xcomp" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+3 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+3)
                else:
                    root_idx = (root_idx[0], root_idx[1]+2)
            elif "VERB-xcomp, VERB-xcomp, DET-det, NOUN-obj, ADP-case" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+5 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+5)
                else:
                    root_idx = (root_idx[0], root_idx[1]+4)

            elif "ADJ-amod, ADP-case" in after_root_pos_dep and splited[0] == "ADJ-amod":
                if root_idx[1]+2 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+2)
                else:
                    root_idx = (root_idx[0], root_idx[1]+1)
            elif "ADV-advmod, ADP-case" in after_root_pos_dep and splited[0] == "ADV-advmod":
                if root_idx[1]+2 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+2)
                else:
                    root_idx = (root_idx[0], root_idx[1]+1)
            elif "ADP-case, NOUN-obj, ADP-case" in after_root_pos_dep and splited[0] == "ADP-case":
                if root_idx[1]+3 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+3)
                else:
                    root_idx = (root_idx[0], root_idx[1]+2)
            elif "ADV-advmod, ADV-advmod, SCONJ-dep" in after_root_pos_dep and splited[0] == "ADV-advmod":
                if root_idx[1]+3 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+3)
                else:
                    root_idx = (root_idx[0], root_idx[1]+2)
            elif "VERB-xcomp" in after_root_pos_dep and splited[0] == "VERB-xcomp":
                if root_idx[1]+1 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+1)
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "ADP-case" in after_root_pos_dep and splited[0] == "ADP-case":
                if root_idx[1]+1 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+1)
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "AUX-cop" in after_root_pos_dep and splited[0] == "AUX-cop":
                if root_idx[1]+1 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+1)
                else:
                    root_idx = (root_idx[0], root_idx[1])
            elif "DET-case" in after_root_pos_dep and splited[0] == "DET-case":
                if root_idx[1]+1 < len(doc_dict) - 1:
                    root_idx = (root_idx[0], root_idx[1]+1)
                else:
                    root_idx = (root_idx[0], root_idx[1])

        j = root_idx[0]
        if root_idx != (None, None):
            while j <= root_idx[1]:
                rel += doc_dict[j]["text"] + " "
                j += 1

            for idx in doc_dict:
                token = doc_dict[idx]["text"]
                if idx < root_idx[0]:
                    arg1 += token + " "
                if idx > root_idx[1]:
                    arg2 += token + " "


        return arg1, rel, arg2


In [38]:
class ArgsRel2:
    def __init__(self):
        self.provavel_rel = []
        self.alinhamentos = []
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")

    def root_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "VERB" and dep == "ROOT") and (idx != 0 and idx != len(doc_dict) - 1):#restringe primeiro e último caracter da frase inteira
                self.provavel_rel.append("VERB")
                return (idx, idx)
        root_idx = self.verb_parse(doc_dict, root_idx)
        return root_idx

    def verb_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "VERB" and (dep == "xcomp" or dep == "acl" or dep == "acl:relacl")) and (idx != 0 and idx != len(doc_dict) - 1):#restringe primeiro e último caracter da frase inteira
                self.provavel_rel.append("VERB")
                return (idx, idx)

        root_idx = self.aux_parse(doc_dict, root_idx)
        return root_idx



    def aux_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "AUX" and dep == "ROOT") and (idx != 0 and idx != len(doc_dict) - 1):#restringe primeiro e último caracter da frase inteira
                self.provavel_rel.append("AUX")
                return (idx, idx)

        root_idx = self.aux_parse2(doc_dict, root_idx)
        return root_idx


    def aux_parse2(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "AUX" and dep == "cop") and (idx != 0 and idx != len(doc_dict) - 1):#restringe primeiro e último caracter da frase inteira
                self.provavel_rel.append("AUX")
                return (idx, idx)
        root_idx = self.noun_parse(doc_dict, root_idx)
        return root_idx

    def noun_parse(self, doc_dict, root_idx):
        #encontra centro da extração pelo root
        for idx in doc_dict:
            pos = doc_dict[idx]["pos"]
            dep = doc_dict[idx]["dep"]
            if (pos == "NOUN" and dep == "ROOT") and (idx != 0 and idx != len(doc_dict) - 1):#restringe primeiro e último caracter da frase inteira
                self.provavel_rel.append("NOUN")
                return (idx, idx)
        return root_idx


    def get_args_rel(self, ext):
        self.alinhamentos = []
        doc = self.nlp(ext)
        doc_dict = {}
        i = 0
        for token in doc:
            doc_dict[i] = {"text": token.text, "pos": token.pos_, "dep": token.dep_}
            i += 1
        root_idx = (None, None)
        self.provavel_rel = []
        root_idx = self.root_parse(doc_dict, root_idx)

        if len(self.provavel_rel)>0 and self.provavel_rel[0] == "VERB":
            if root_idx[0]-1 != 0:
                if doc_dict[root_idx[0]-1]["pos"] in ["AUX",'ADV']:
                    root_idx = (root_idx[0]-1, root_idx[1])

        #verificando elementos que compoem a rel depois do centro
        if root_idx != (None, None):
            for j in range(root_idx[1]+1, len(doc_dict)):
                pos = doc_dict[j]["pos"]
                self.provavel_rel.append(pos)

        adp_idxs = []
        for idx, pos_ in enumerate(self.provavel_rel[1:-1]):
            if pos_ in ['ADJ','ADV','NOUN', 'VERB','ADV']:
                continue
            elif pos_ == 'ADP':
                adp_idxs.append(idx+1)
                continue
            else:
                break
        adp_idxs.append(0)

        for idx in adp_idxs:
            arg1 = ""
            rel = ""
            arg2 = ""
            if root_idx != (None, None):
                new_root_idx = (root_idx[0],root_idx[1]+idx)
                j = new_root_idx[0]
                while j <= new_root_idx[1]:
                    rel += doc_dict[j]["text"] + " "
                    j += 1

                for idx in doc_dict:
                    token = doc_dict[idx]["text"]
                    if idx < new_root_idx[0]:
                        arg1 += token + " "
                    if idx > new_root_idx[1]:
                        arg2 += token + " "

            self.alinhamentos.append((arg1,rel,arg2))


        return self.alinhamentos

In [475]:
"""
Novo metodo de alinhamento.

pega a ext inteira e a sentenca
transforma em tokens
rels = []
gero diferentes relações (combinação):
	Combinação:
		loop (n=1):
		  a primeira relacao envolve [1:len(tokens)-2](rel)
		  a segunda em diante, deve ser [n:len(tokens)-(n+1)](rel)
		  rels.append(rel)
		  stop:
		    quando n >= len(tokens)-(n+1)

para cada relação diferente busco todos os matches que podem ocorrer na sentença(percorre tudo)
se encontrar match em uma relação, armazenar combinação como valida.
"""
class Alignment():
    def __init__(self):
        self.provavel_rel = []
        self.alinhamentos = []
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")

    #TODO impolementar todos tipos de alinhamentos em uma unica função



class ArgsRel3:
    def __init__(self):
        self.provavel_rel = []
        self.alinhamentos = []
        self.matcher = OIE_Match()
        try:
            self.nlp = spacy.load("pt_core_news_lg")
        except:
            os.system("python -m spacy download pt_core_news_lg")
            self.nlp = spacy.load("pt_core_news_lg")

    def get_args_rel(self, ext, sent):
        self.alinhamentos = []
        pos = []
        rels = []
        ext_list = ext.split(" ")
        sent_doc = self.nlp(sent)


        #permutando relações
        for i in range(1,len(ext_list)):
            for j in range(i+1,len(ext_list)):
                if i == len(ext_list)-1:
                    rel = ext_list[i]
                else:
                    rel = ext_list[i:j]
                if (rel,(i,j)) not in rels:
                    rels.append((rel,(i,j)))

        rels.sort(key = lambda x: len(x[0]), reverse=True)

        for rel in rels:
            idx = rel[1]
            sent_list = sent.split(" ")
            ext_list = ext.split(" ")
            arg0 = " ".join(ext_list[:idx[0]])
            arg1 = " ".join(ext_list[idx[1]:len(sent_list)])
            rel = " ".join(rel[0])

            valid = self.matcher.match(sent, arg0, rel, arg1)
            if valid[3]:
                #colhe pos da relação do alinhamento, o pos usado é o da sent nos tokens da ext
                aux = []
                aux_dep = []
                cur_ext = []
                cur_dep = []
                for span in valid[:-1]:
                    span_tk = sent_doc[span[0]:span[1]+1]
                    for token in span_tk:
                        aux.append(token.pos_)
                        aux_dep.append(token.dep_)
                    cur_ext.append(aux)
                    cur_dep.append(aux_dep)
                    aux = []
                    aux_dep = []
                pos.append(((arg0,rel,arg1),cur_ext, cur_dep))
        #utiliza regras no pos da relação para filtrar alinhamentos
                ali_gerado = ((arg0,rel,arg1),cur_ext, cur_dep)
                #print(ali_gerado)
                rel_pos = ali_gerado[1][1]
                rel_dep = ali_gerado[2][1]
                #print(rel_pos)
                #print(rel_dep)
                inicio = [[rel_pos[0], rel_dep[0]]]
                meio = []
                #print(rel_pos, rel_dep)
                for x,y in zip(rel_pos[1:-1], rel_dep[1:-1]):
                    meio.append([x,y])
                fim = [[rel_pos[-1], rel_dep[-1]]]
                #print(inicio)
                #print(meio)
                #print(fim)

                first = False
                middle = False
                middle_counter = 0
                #inicio
                for i,tags in enumerate(inicio):
                    p_tag = tags[0]
                    p_dep = tags[1]
                    if p_tag == "AUX" and i == 0:
                        first = True
                        if len(rel_pos) == 1:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                    elif (p_tag == "VERB" and p_dep == "ROOT") and i == 0:
                        first = True
                        if len(rel_pos) == 1:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                    elif p_tag == "VERB" and i == 0:
                        first = True
                        if len(rel_pos) == 1:
                            self.alinhamentos.append(ali_gerado[0])
                            return self.alinhamentos
                #meio
                for i,tags in enumerate(meio):
                        #print(tags)
                        p_tag = tags[0]
                        #print(p_tag)
                        if p_tag in ['ADJ','NOUN', 'VERB', "AUX","DET"] and first:
                            middle_counter += 1
                if middle_counter == len(meio):
                    middle = True

                #fim
                for i,tags in enumerate(fim):
                    p_tag = tags[0]
                    if len(rel_pos) == 2 and p_tag == "VERB" and first:
                        self.alinhamentos.append(ali_gerado[0])
                        return self.alinhamentos
                    elif len(rel_pos) == 2 and p_tag == "AUX" and first:
                        self.alinhamentos.append(ali_gerado[0])
                        return self.alinhamentos
                    elif len(rel_pos) == 2 and p_tag == "ADP" and first:
                        self.alinhamentos.append(ali_gerado[0])
                        return self.alinhamentos
                    elif len(rel_pos) > 2 and p_tag == "ADP" and first and middle:
                        self.alinhamentos.append(ali_gerado[0])
                        return self.alinhamentos

        if len(self.alinhamentos) == 0:
            self.alinhamentos.append((" "," "," "))

        return self.alinhamentos

In [476]:
ali = ArgsRel3()
ali.get_args_rel("Alan é filho de o pai dele", "Alan é filho de o pai dele")
#ali.alinhamentos

[('Alan', 'é filho de', 'o pai dele')]

In [402]:
ar = ArgsRel2()
ar.get_args_rel('o SNS estava agora sob enorme pressão')

NameError: name 'ArgsRel2' is not defined

In [281]:
valid = {}
id = 0
with open("development/100-gold.txt", "r", encoding="utf-8") as f:
    lines = f.read()
    samples = lines.split("\n\n")
for ext in samples:
    sent = ""
    arg0 = ""
    rel = ""
    arg1 = ""
    ext_lines = ext.split("\n")
    if ext_lines != ['']:
        for line in ext_lines:
            line = line.split("\t")
            sent += line[0] + " "
            if "ARG0" in line[8]:
                arg0+=line[0] + " "
            if "V" in line[8]:
                rel+=line[0] + " "
            if "ARG1" in line[8]:
                arg1+=line[0] + " "
        valid[id] = {"ext": arg0+" "+rel+" "+arg1,"sent": sent, "arg1": transform_portuguese_contractions(arg0), "rel": transform_portuguese_contractions(rel), "arg2": transform_portuguese_contractions(arg1)}
        id+=1

In [282]:
valid

{0: {'ext': 'um casal  poderia abrir  quatro contas Tesco ',
  'sent': 'Teoricamente , um casal poderia abrir quatro contas Tesco e ganhar 3 % em £ 12,000 – £ 360 . ',
  'arg1': 'um casal',
  'rel': 'poderia abrir',
  'arg2': 'quatro contas Tesco'},
 1: {'ext': 'isso  é  cada vez mais digno de atenção ',
  'sent': 'Um avanço rápido para 2016 e isso é cada vez mais digno de atenção . ',
  'arg1': 'isso',
  'rel': 'é',
  'arg2': 'cada vez mais digno de atenção'},
 2: {'ext': 'as nuvens de chuvas ácidas  podem ter  um lado positivo ',
  'sent': 'Surpreendentemente como pode parecer , até as nuvens de chuvas ácidas podem ter um lado positivo . ',
  'arg1': 'as nuvens de chuvas ácidas',
  'rel': 'podem ter',
  'arg2': 'um lado positivo'},
 3: {'ext': 'O mundo  pode estar  absurdo ',
  'sent': 'O mundo pode estar enfurecido e absurdo - em o entanto por o menos alguém tem dignidade para continuar a protestar contra esse fato . ',
  'arg1': 'O mundo',
  'rel': 'pode estar',
  'arg2': 'absurdo'

In [477]:
arg2 = ArgsRel3()
nlp = spacy.load("pt_core_news_lg")
correct = 0
incorrect = 0
correct_dict = {}
incorrect_dict = {}
json_dict = {}
rel_all = []
for id in valid:
    sentence = valid[id]["sent"]
    sentence = transform_portuguese_contractions(sentence)
    ext = valid[id]["ext"]
    ext = transform_portuguese_contractions(ext)
    print("-"*50)
    print("ext: ",ext)
    rel_idx = (len(valid[id]["arg1"].split(" ")), len(valid[id]["arg1"].split(" ")) + len(valid[id]["rel"].split(" ")) - 1)
    rel_text = valid[id]["rel"]
    rel_local = []
    rel_pos = []
    rel_dep = []
    before_root = []
    root_list = []
    after_root = []
    rel_list = [ext, rel_text]

    ext2 = nlp(ext)
    root = False
    for token in ext2:
        #search root, before and after root
        if token.i >= rel_idx[0] and token.i <= rel_idx[1]:
            if token.dep_ != "ROOT" and root == False:
                before_root.append("-".join([token.pos_, token.dep_]))
            if token.dep_ == "ROOT":
                root_list.append("-".join([token.pos_, token.dep_]))
                root = True
            if token.dep_ != "ROOT" and root == True:
                after_root.append("-".join([token.pos_, token.dep_]))

    after_root = ", ".join(after_root)
    before_root = ", ".join(before_root)
    root_list = ", ".join(root_list)

    if len(before_root) == 0:
        rel_list.append("None")
    else:
        rel_list.append(before_root)
    if len(root_list) == 0:
        rel_list.append("None")
    else:
        rel_list.append(root_list)
    if len(after_root) == 0:
        rel_list.append("None")
    else:
        rel_list.append(after_root)

    tk_sint = []
    for token in ext2:
        print(token.text, token.pos_, token.dep_)
        if token.i >= rel_idx[0] and token.i <= rel_idx[1]:
            rel_pos.append(token.pos_)
            rel_dep.append(token.dep_)
            tk_sint.append("-".join([token.pos_, token.dep_]))
    rel_list.append(", ".join(tk_sint))
    rel_all.append(rel_list)

    alinhamentos = arg2.get_args_rel(ext, sentence)
    print("alinhamentos: ", alinhamentos)
    alinhamentos = [alinhamentos[0]]
    n_valid = False
    for alinhamento in alinhamentos:
        arg0, rel, arg1 = alinhamento

        arg0 = clean_extraction(arg0)
        rel = clean_extraction(rel)
        arg1 = clean_extraction(arg1)
        try:
            if arg0[-1] == " ":
                arg0 = arg0[:-1]
            if rel[-1] == " ":
                rel = rel[:-1]
            if arg1[-1] == " ":
                arg1 = arg1[:-1]
        except:
            pass

        true_arg0 = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["arg1"])]))
        true_rel = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["rel"])]))
        true_arg1 = clean_extraction(" ".join([str(token) for token in nlp(valid[id]["arg2"])]))
        print("alignment: ", (arg0, rel, arg1))
        if arg0 == true_arg0 and rel == true_rel and arg1 == true_arg1:
            correct_dict[id] = {"ext": ext, "arg0": arg0, "rel": rel, "arg1": arg1}
            json_dict[correct] = {"ID": id, "sent": valid[id]["sent"], "ext": [{
                "arg1": valid[id]["arg1"],
                "rel": valid[id]["rel"],
                "arg2": valid[id]["arg2"]
            }]}
            correct += 1
            n_valid = True
    if not n_valid:
        incorrect += 1
        incorrect_dict[id] = {"ext": ext, "alignment":{"arg0": arg0, "rel": rel, "arg1": arg1}, "expected": {"arg0": true_arg0, "rel": true_rel, "arg1": true_arg1}}
    print("expected: ", (valid[id]["arg1"], valid[id]["rel"], valid[id]["arg2"]))
    print("total: ", correct+incorrect)
    print("correct: ", correct)
    print("incorrect: ", incorrect)
    #print("expected_rel_pos: ", rel_pos)
    #print("expected_rel_dep: ", rel_dep)
    #print("before_root: ", before_root)
    #print("root_list: ", root_list)
    #print("after_root: ", after_root)

--------------------------------------------------
ext:  um casal poderia abrir quatro contas Tesco
um DET det
casal NOUN nsubj
poderia VERB ROOT
abrir VERB xcomp
quatro NUM nummod
contas NOUN obj
Tesco PROPN appos
alinhamentos:  [('um casal', 'poderia abrir', 'quatro contas Tesco')]
alignment:  ('um casal', 'poderia abrir', 'quatro contas Tesco')
expected:  ('um casal', 'poderia abrir', 'quatro contas Tesco')
total:  1
correct:  1
incorrect:  0
--------------------------------------------------
ext:  isso é cada vez mais digno de atenção
isso PRON nsubj
é AUX cop
cada PROPN nmod
vez ADP case
mais ADV nmod
digno ADJ ROOT
de ADP case
atenção NOUN obl
alinhamentos:  [('isso', 'é', 'cada vez mais digno de atenção')]
alignment:  ('isso', 'é', 'cada vez mais digno de atenção')
expected:  ('isso', 'é', 'cada vez mais digno de atenção')
total:  2
correct:  2
incorrect:  0
--------------------------------------------------
ext:  as nuvens de chuvas ácidas podem ter um lado positivo
as DET det


In [24]:
valid = {}
id = 0
with open("development/200-silver.txt", "r", encoding="utf-8") as f:
    lines = f.read()
    samples = lines.split("\n\n")
for ext in samples:
    sent = ""
    arg0 = ""
    rel = ""
    arg1 = ""
    ext_lines = ext.split("\n")
    if ext_lines != ['']:
        for line in ext_lines:
            line = line.split("\t")
            sent += line[0] + " "
            if "ARG0" in line[8]:
                arg0+=line[0] + " "
            if "V" in line[8]:
                rel+=line[0] + " "
            if "ARG1" in line[8]:
                arg1+=line[0] + " "
        valid[id] = {"ext": arg0+" "+rel+" "+arg1,"sent": sent, "arg1": transform_portuguese_contractions(arg0), "rel": transform_portuguese_contractions(rel), "arg2": transform_portuguese_contractions(arg1)}
        id+=1

In [25]:
from OIE.datasets.src.match import OIE_Match
import pathlib
import json
path = "saida_match"
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
with open("saida_match/json_dump.json", "a", encoding="utf-8") as f:
    f.write(json.dumps(json_dict))

oie_match = OIE_Match("200-silver", "saida_match")
oie_match.run(sequential=True)

Carregando dados: 100%|██████████| 158/158 [00:01<00:00, 92.26it/s]


initial samples:  158 || valid samples:  98


Criando conll: 100%|██████████| 98/98 [00:00<00:00, 192.31it/s]


In [54]:
with open("saida_match/json_dump.json", "r", encoding="utf-8") as f:
    file = json.load(f)
for key in range(len(file)):
    match = eng.match(file[str(key)]["sent"], file[str(key)]["ext"][0]["arg1"],file[str(key)]["ext"][0]["rel"],file[str(key)]["ext"][0]["arg2"])

EXTRAÇÃO VÁLIDA sent: Para aqueles que seguem as transições de as redes sociais em o Capitol Hill, esta será um pouco diferente. arg1: [aqueles] rel: [seguem] arg2: [as, transições, de, as, redes, sociais]
EXTRAÇÃO VÁLIDA sent: Para aqueles que seguem as transições de as redes sociais em o Capitol Hill, esta será um pouco diferente. arg1: [esta] rel: [será] arg2: [um, pouco, diferente]
EXTRAÇÃO VÁLIDA sent: Mas em uma pausa de a sua retórica passada acerca de a restrição de a imigração, o nomeado de a GOP anunciou que, como presidente, ele iria permitir "números tremendos" de imigrantes legais, baseado em um "sistema de mérito". arg1: [ele] rel: [iria, permitir] arg2: [números, tremendos]
EXTRAÇÃO VÁLIDA sent: Os novos gastos são alimentados por a grande conta bancária de Clinton. arg1: [os, novos, gastos] rel: [são, alimentados, por] arg2: [a, grande, conta, bancária, de, clinton]
EXTRAÇÃO VÁLIDA sent: No início de outubro, a equipe de transição utilizou o mesmo local para se encontra

In [15]:
arg2 = ArgsRel2()
nlp = spacy.load("pt_core_news_lg")
ext = "Alan estava indo dançando em a rua."
ext = clean_extraction(ext)
ext = transform_portuguese_contractions(ext)
sent = nlp(ext)
print("token - pos - dep")
print("-"*50)
for token in sent:
        print(token.text,"-",token.pos_,"-", token.dep_)
print("------------------------------------------------")
print("sent: ",sent)
print("alignment: ", arg2.get_args_rel(ext))


token - pos - dep
--------------------------------------------------
Alan - PROPN - nsubj
estava - AUX - aux
indo - VERB - ROOT
dançando - VERB - advcl
em - ADP - case
a - DET - det
rua - NOUN - obl
------------------------------------------------
sent:  Alan estava indo dançando em a rua
alignment:  [('Alan ', 'estava indo dançando em ', 'a rua '), ('Alan ', 'estava indo ', 'dançando em a rua ')]
